<a href="https://colab.research.google.com/github/surabhipandey18/FoodAgri/blob/main/fertilizer_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
from IPython import get_ipython
from IPython.display import display
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, Dense, Dropout
from tensorflow.keras.metrics import CategoricalAccuracy

In [63]:
df = pd.read_csv('/content/drive/MyDrive/Hackathon/f2.csv')
df.head()

,Temparature,Humidity,Moisture,Soil_Type,Crop_Type,Nitrogen,Potassium,Phosphorous,Fertilizer
0,20,83,26,Clayey,rice,90,49,36,Urea
1,25,84,32,Loamy,rice,66,59,36,Urea
2,33,64,50,Loamy,Wheat,41,0,0,Urea
3,34,65,54,Loamy,Wheat,38,0,0,Urea
4,38,72,51,Loamy,Wheat,39,0,0,Urea


In [88]:
df.shape

(552, 9)

In [64]:
df = df.dropna()

In [65]:
categorical_col = ['Soil_Type', 'Crop_Type']
numerical_col = ['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']
target = ['Fertilizer']

In [66]:
std = StandardScaler()
encoder = OneHotEncoder()

In [67]:
target = encoder.fit_transform(df[target]).toarray()

In [68]:
preprocess = ColumnTransformer([
    ('std', std, numerical_col),
    ('encoder', encoder, categorical_col)
])

In [92]:
data = preprocess.fit_transform(df)
data

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4416 stored elements and shape (552, 28)>

In [85]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

In [90]:
X_test.shape

(111, 28)

In [98]:
output_classes = y_train.shape[1]
model = models.Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1),
           kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.5),
    layers.Flatten(),
    layers.Dense(output_classes, activation='softmax'),
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[CategoricalAccuracy()])
tf.config.run_functions_eagerly(True)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 26, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 26, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1664)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_68 (Dense)                │ (None, 14)             │        23,310 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,566 (92.05 KB)

 Trainable params: 23,566 (92.05 KB)

 Non-trainable params: 0 (0.00 B)

In [100]:
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

Epoch 1/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - categorical_accuracy: 0.9617 - loss: 0.1939 - val_categorical_accuracy: 0.9775 - val_loss: 0.1962
Epoch 2/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - categorical_accuracy: 0.9610 - loss: 0.1913 - val_categorical_accuracy: 0.9775 - val_loss: 0.1864
Epoch 3/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - categorical_accuracy: 0.9727 - loss: 0.1780 - val_categorical_accuracy: 0.9775 - val_loss: 0.1820
Epoch 4/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - categorical_accuracy: 0.9715 - loss: 0.1704 - val_categorical_accuracy: 0.9775 - val_loss: 0.1726
Epoch 5/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - categorical_accuracy: 0.9643 - loss: 0.1820 - val_categorical_accuracy: 0.9775 - val_loss: 0.1705
Epoch 6/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - categorical_accuracy: 0.9460 - loss: 0.1897 - val_categorical_accuracy: 0.9775 - val_loss: 0.1660
Epoch 7/30
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - categorical_accuracy: 0.9660 - loss: 0.

In [101]:
accuracy = model.evaluate(X_test, y_test)[1]
print(f"Test Accuracy: {accuracy * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - categorical_accuracy: 0.9813 - loss: 0.0969
Test Accuracy: 98.20%


In [102]:
model.save('fertilizer_recommendation.keras')